In [1]:
import xarray as xr

biostates = xr.open_dataset('/cluster/projects/itk-SINMOD/coral-mapping/midnor/BioStates_2019.nc')

biostates

<xarray.Dataset> Size: 503GB
Dimensions:               (time: 365, dummySampleDim: 6, zc: 50, xc: 950,
                           yc: 555)
Coordinates:
    time                  (time, dummySampleDim) int32 9kB ...
  * xc                    (xc) float32 4kB 800.0 1.6e+03 ... 7.592e+05 7.6e+05
  * yc                    (yc) float32 2kB 800.0 1.6e+03 ... 4.432e+05 4.44e+05
  * zc                    (zc) float32 200B 3.0 6.0 10.0 ... 2.75e+03 3e+03
Dimensions without coordinates: dummySampleDim
Data variables: (12/21)
    grid_mapping          int32 4B ...
    LayerDepths           (zc) float32 200B ...
    depth                 (yc, xc) float32 2MB ...
    DXxDYy                (yc, xc) float32 2MB ...
    nitrate               (time, zc, yc, xc) float32 38GB ...
    silicate              (time, zc, yc, xc) float32 38GB ...
    ...                    ...
    detritus_fast         (time, zc, yc, xc) float32 38GB ...
    DOC                   (time, zc, yc, xc) float32 38GB ...
    cDOM                  (time, zc, yc, xc) float32 38GB ...
    silicate_detritus     (time, zc, yc, xc) float32 38GB ...
    sediment_Si           (time, yc, xc) float32 770MB ...
    sediment_N            (time, yc, xc) float32 770MB ...
Attributes: (12/31)
    Conventions:                            CF-1.5
    title:                                  Current data from SINMOD
    institution:                            SINTEF Ocean
    grid_mapping:                           grid_mapping
    grid_mapping_name:                      polar_stereographic
    straight_vertical_longitude_from_pole:  58.0
    ...                                     ...
    CH:                                     0.3
    CI:                                     5.0
    icedyn:                                 F
    tidal_components:                       [0 0 0 0 0 0 0 0 0 0 0]
    history:                                Thu Nov  7 07:52:44 2024: ncrcat ...
    NCO:                                    netCDF Operators version 5.1.3 (H...

In [2]:
import dask
from dask.distributed import Client

client = Client()
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [ ]:
target_variables = ['diatoms', 'flagellates', 'ciliates', 'HNANO', 'bacteria', 'calanus_finmarchicus', 'calanus_glacialis','detritus_slow', 'detritus_fast',
                    'nitrate','silicate','ammonium','DOC','cDOM', 'silicate_detritus','sediment_Si','sediment_N']

biostates_surface = biostates[target_variables].isel(zc=0)

In [4]:
# Enable Dask by chunking the dataset (spatial and temporal dimensions)
biostates_surface = biostates_surface.chunk({'time': 10, 'xc': 100, 'yc': 100})

# Calculate the global min and max for each variable across all dimensions
min_vals = biostates_surface.min(dim=("time", "xc", "yc"))
max_vals = biostates_surface.max(dim=("time", "xc", "yc"))

# Apply normalisation (broadcasting min and max across the dataset)
biostates_surface_normalised = (biostates_surface - min_vals) / (max_vals - min_vals)

# Print the resulting normalised dataset
biostates_surface_normalised.to_netcdf('/cluster/home/haroldh/spGDMM/1_data/1_raw/biostates_surface_normalised.nc')

/cluster/home/haroldh/.conda/envs/spGDMM/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 67.48 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/cluster/home/haroldh/.conda/envs/spGDMM/lib/python3.12/site-packages/dask/_task_spec.py:745: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/cluster/home/haroldh/.conda/envs/spGDMM/lib/python3.12/site-packages/dask/_task_spec.py:745: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/cluster/home/haroldh/.conda/envs/spGDMM/lib/python3.12/site-packages/dask/_task_spec.py:745: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/cluster/home/haroldh/.conda

In [1]:
import xarray as xr

biomod = xr.open_dataset('/cluster/home/haroldh/spGDMM/1_data/1_raw/biostates_surface_normalised.nc')

# Assert data is as expected
assert biomod.to_array().min().item() >= 0
assert biomod.to_array().max().item() <= 1

In [ ]:
# Assert normalised dataset is as expected